# Xarray

`xarray` (antigamente conhecido como xray) é um pacote Python que faz trabalhar com arrays multi-dimensionais simples, efficente e divertido!

Arrays multi-dimensionais são uma parte essencial em computação científica. Eles são encontrados em uma grande variedade de áreas como física, astronômia, geociências, e etc. No Python, NumPy proporciona a estrutura fundamental para trabalhar com arrays multi-dimensionais. No entanto, geralmente os conjuntos de dados são mais do que números puros; eles possuem nomes (labels) que encorporam como o array é mapeado no espaço, tempo, etc.

Introduzindo dimensões, coordenadas e atributos em cima de arrays NumPy, xarray é capaz de entender esses labels e usa-lo para proporcionar uma experiência mais intuitiva, concisa e menos proícia para erros. Xarray também proporciona uma variedade de funções para análise de dados e visualização. Xarray foi inspirado e usa muito do que foi desenvolvido para o pandas. Xarray pode ler e escrever dados dos formatos mais comuns e é particularmente feito para funcionar bem com arquivos netCDF.